In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "./data/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import collections
print(os.listdir("./data/"))

['Cleaned_AD_and_Scalp_Care']


In [4]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In [5]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling

W0709 14:01:52.781745 140073948907264 deprecation_wrapper.py:119] From /home/dp8772/.local/lib/python3.7/site-packages/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [6]:
BERT_VOCAB= './bert/bert/models/uncased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = './bert/bert/models/uncased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = './bert/bert/models/uncased_L-12_H-768_A-12/bert_config.json'

In [7]:
tokenization.validate_case_matches_checkpoint(True,BERT_INIT_CHKPNT)
tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=True)

W0709 14:01:58.023617 140073948907264 deprecation_wrapper.py:119] From /home/dp8772/.local/lib/python3.7/site-packages/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [8]:
train_data_path='/home/dp8772/data/Cleaned_AD_and_Scalp_Care/train2.csv'
train = pd.read_csv(train_data_path)

In [9]:
train.head()

,id,verbatim,Hair Loss,Viscosity,Environmental,Smell,Length,Formula Change,Scalp Care,Promotions,...,Returning Customer,Instructions,Shine,Price or Value for Money,Volume & Fullness,Design,Portability,Lather,Spray Application,Professional Care
0,0,This shampoo sprays crazy good and it has save...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,My teenaged daughter has been using Head & Sho...,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,2,"To my surprise, Dove DermaCare Scalp Invigorat...",0,0,0,1,1,0,1,0,...,1,0,0,1,0,0,0,0,1,0
3,3,I let my boyfriend try this product. He loves ...,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,4,"Its conveniently packaged, but if this bottle ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [10]:
ID = 'id'
DATA_COLUMN = 'verbatim'
LABEL_COLUMNS = ['Hair Loss','Viscosity','Environmental','Smell','Length','Formula Change','Scalp Care','Promotions','Ingredients','Quality','Cleanliness','Item Availability','Gift Purposes','Texture','Hair Dryness','Thickness','Ease of Use','Time or Frequency of Use','Customer Wishlist','Look & Feel','2in1','Overall Satisfaction','Discontinued Products','Styling','Treatment Time','Hair Type','Color Treated Hair','Manageable','Shipping','Protection','Loyalty','Recommended','Size','Customer Service','Oily Greasy','Residue','Kids Use','Nutrition & Hair Health','Split Ends & Breakage','Soft & Smooth','Meets Expectations','Results','Economical','Weather','Packaging','Returning Customer','Instructions','Shine','Price or Value for Money','Volume & Fullness','Design','Portability','Lather','Spray Application','Professional Care']

In [11]:
print (len(LABEL_COLUMNS))

55


In [12]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, labels=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            labels: (Optional) [string]. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids, is_real_example=True):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids,
        self.is_real_example=is_real_example

In [13]:
def create_examples(df, labels_available=True):
    """Creates examples for the training and dev sets."""
    examples = []
    for (i, row) in enumerate(df.values):
        guid = row[0]
        text_a = row[1]
        if labels_available:
            labels = row[2:]
        else:
            labels = [0,0,0,0,0,0]
        examples.append(
            InputExample(guid=guid, text_a=text_a, labels=labels))
    return examples

In [14]:
TRAIN_VAL_RATIO = 0.8
TEST_RATIO=0.1
LEN = train.shape[0]
SIZE_TRAIN = int(TRAIN_VAL_RATIO*LEN)
SIZE_TEST = int(TEST_RATIO*LEN)
print (SIZE_TRAIN)
x_train = train[:SIZE_TRAIN]
x_val = train[SIZE_TRAIN:SIZE_TRAIN+SIZE_TEST]
x_test= train[SIZE_TRAIN+SIZE_TEST:]

# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_examples = create_examples(x_train)

7452


In [15]:
train.shape, x_train.shape, x_val.shape,x_test.shape

((9315, 57), (7452, 57), (931, 57), (932, 57))

In [16]:
import pandas

def convert_examples_to_features(examples,  max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    features = []
    for (ex_index, example) in enumerate(examples):
        print(example.text_a)
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        
        labels_ids = []
        for label in example.labels:
            labels_ids.append(int(label))

        if ex_index < 0:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %s)" % (example.labels, labels_ids))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_ids=labels_ids))
    return features

In [17]:
MAX_SEQ_LENGTH = 256

In [18]:
def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
    """Creates a classification model."""
    model = modeling.BertModel(
        config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)

    # In the demo, we are doing a simple classification task on the entire
    # segment.
    #
    # If you want to use the token-level output, use model.get_sequence_output()
    # instead.
    output_layer = model.get_pooled_output()

    hidden_size = output_layer.shape[-1].value

    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
        "output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):
        if is_training:
            # I.e., 0.1 dropout
            output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        
        # probabilities = tf.nn.softmax(logits, axis=-1) ### multiclass case
        probabilities = tf.nn.sigmoid(logits)#### multi-label case
        
        labels = tf.cast(labels, tf.float32)
        tf.logging.info("num_labels:{};logits:{};labels:{}".format(num_labels, logits, labels))
        per_example_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)
        loss = tf.reduce_mean(per_example_loss)

        # probabilities = tf.nn.softmax(logits, axis=-1)
        # log_probs = tf.nn.log_softmax(logits, axis=-1)
        #
        # one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
        #
        # per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        # loss = tf.reduce_mean(per_example_loss)

        return (loss, per_example_loss, logits, probabilities)


def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
    """Returns `model_fn` closure for TPUEstimator."""

    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        #tf.logging.info("*** Features ***")
        #for name in sorted(features.keys()):
        #    tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]
        is_real_example = None
        if "is_real_example" in features:
             is_real_example = tf.cast(features["is_real_example"], dtype=tf.float32)
        else:
             is_real_example = tf.ones(tf.shape(label_ids), dtype=tf.float32)

        is_training = (mode == tf.estimator.ModeKeys.TRAIN)

        (total_loss, per_example_loss, logits, probabilities) = create_model(
            bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
            num_labels, use_one_hot_embeddings)

        tvars = tf.trainable_variables()
        initialized_variable_names = {}
        scaffold_fn = None
        if init_checkpoint:
            (assignment_map, initialized_variable_names
             ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
            if use_tpu:

                def tpu_scaffold():
                    tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
                    return tf.train.Scaffold()

                scaffold_fn = tpu_scaffold
            else:
                tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

        tf.logging.info("**** Trainable Variables ****")
        for var in tvars:
            init_string = ""
            if var.name in initialized_variable_names:
                init_string = ", *INIT_FROM_CKPT*"
            #tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,init_string)

        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:

            train_op = optimization.create_optimizer(
                total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                train_op=train_op,
                scaffold=scaffold_fn)
        elif mode == tf.estimator.ModeKeys.EVAL:

            def metric_fn(per_example_loss, label_ids, probabilities, is_real_example):

                logits_split = tf.split(probabilities, num_labels, axis=-1)
                label_ids_split = tf.split(label_ids, num_labels, axis=-1)
                # metrics change to auc of every class
                eval_dict = {}
                for j, logits in enumerate(logits_split):
                    label_id_ = tf.cast(label_ids_split[j], dtype=tf.int32)
                    current_auc, update_op_auc = tf.metrics.auc(label_id_, logits)
                    eval_dict[str(j)] = (current_auc, update_op_auc)
                eval_dict['eval_loss'] = tf.metrics.mean(values=per_example_loss)
                return eval_dict

                ## original eval metrics
                # predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
                # accuracy = tf.metrics.accuracy(
                #     labels=label_ids, predictions=predictions, weights=is_real_example)
                # loss = tf.metrics.mean(values=per_example_loss, weights=is_real_example)
                # return {
                #     "eval_accuracy": accuracy,
                #     "eval_loss": loss,
                # }

            eval_metrics = metric_fn(per_example_loss, label_ids, probabilities, is_real_example)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                eval_metric_ops=eval_metrics,
                scaffold=scaffold_fn)
        else:
            print("mode:", mode,"probabilities:", probabilities)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                predictions={"probabilities": probabilities},
                scaffold=scaffold_fn)
        return output_spec

    return model_fn

In [19]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 20.0

# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500

In [20]:
OUTPUT_DIR = "./output"
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    keep_checkpoint_max=1,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [21]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
    all_input_ids = []
    all_input_mask = []
    all_segment_ids = []
    all_label_ids = []

    for feature in features:
        all_input_ids.append(feature.input_ids)
        all_input_mask.append(feature.input_mask)
        all_segment_ids.append(feature.segment_ids)
        all_label_ids.append(feature.label_ids)

    def input_fn(params):
        batch_size = params["batch_size"]

        num_examples = len(features)

        # This is for demo purposes and does NOT scale to large data sets. We do
        # not use Dataset.from_generator() because that uses tf.py_func which is
        # not TPU compatible. The right way to load data is with TFRecordReader.
        d = tf.data.Dataset.from_tensor_slices({
            "input_ids":
                tf.constant(
                    all_input_ids, shape=[num_examples, seq_length],
                    dtype=tf.int32),
            "input_mask":
                tf.constant(
                    all_input_mask,
                    shape=[num_examples, seq_length],
                    dtype=tf.int32),
            "segment_ids":
                tf.constant(
                    all_segment_ids,
                    shape=[num_examples, seq_length],
                    dtype=tf.int32),
            "label_ids":
                tf.constant(all_label_ids, shape=[num_examples, len(LABEL_COLUMNS)], dtype=tf.int32),
        })

        if is_training:
            d = d.repeat()
            d = d.shuffle(buffer_size=100)

        d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
        return d

    return input_fn

In [22]:
class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
    When running eval/predict on the TPU, we need to pad the number of examples
    to be a multiple of the batch size, because the TPU requires a fixed batch
    size. The alternative is to drop the last batch, which is bad because it means
    the entire output data won't be generated.
    We use this class instead of `None` because treating `None` as padding
    battches could cause silent errors.
    """

In [23]:
def convert_single_example(ex_index, example, max_seq_length,
                           tokenizer):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        return InputFeatures(
            input_ids=[0] * max_seq_length,
            input_mask=[0] * max_seq_length,
            segment_ids=[0] * max_seq_length,
            label_ids=0,
            is_real_example=False)

    tokens_a = tokenizer.tokenize(example.text_a)
    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)

    if tokens_b:
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[0:(max_seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0     0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    if tokens_b:
        for token in tokens_b:
            tokens.append(token)
            segment_ids.append(1)
        tokens.append("[SEP]")
        segment_ids.append(1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    labels_ids = []
    for label in example.labels:
        labels_ids.append(int(label))


    feature = InputFeatures(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        label_ids=labels_ids,
        is_real_example=True)
    return feature


def file_based_convert_examples_to_features(
        examples, max_seq_length, tokenizer, output_file):
    """Convert a set of `InputExample`s to a TFRecord file."""

    writer = tf.python_io.TFRecordWriter(output_file)

    for (ex_index, example) in enumerate(examples):
        #if ex_index % 10000 == 0:
            #tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

        feature = convert_single_example(ex_index, example,
                                         max_seq_length, tokenizer)

        def create_int_feature(values):
            f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
            return f

        features = collections.OrderedDict()
        features["input_ids"] = create_int_feature(feature.input_ids)
        features["input_mask"] = create_int_feature(feature.input_mask)
        features["segment_ids"] = create_int_feature(feature.segment_ids)
        features["is_real_example"] = create_int_feature(
            [int(feature.is_real_example)])
        if isinstance(feature.label_ids, list):
            label_ids = feature.label_ids
        else:
            label_ids = feature.label_ids[0]
        features["label_ids"] = create_int_feature(label_ids)

        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())
    writer.close()


def file_based_input_fn_builder(input_file, seq_length, is_training,
                                drop_remainder):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""

    name_to_features = {
        "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
        "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "label_ids": tf.FixedLenFeature([55], tf.int64),
        "is_real_example": tf.FixedLenFeature([], tf.int64),
    }

    def _decode_record(record, name_to_features):
        """Decodes a record to a TensorFlow example."""
        example = tf.parse_single_example(record, name_to_features)

        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.to_int32(t)
            example[name] = t

        return example

    def input_fn(params):
        """The actual input function."""
        batch_size = params["batch_size"]

        # For training, we want a lot of parallel reading and shuffling.
        # For eval, we want no shuffling and parallel reading doesn't matter.
        d = tf.data.TFRecordDataset(input_file)
        if is_training:
            d = d.repeat()
            d = d.shuffle(buffer_size=100)

        d = d.apply(
            tf.contrib.data.map_and_batch(
                lambda record: _decode_record(record, name_to_features),
                batch_size=batch_size,
                drop_remainder=drop_remainder))

        return d

    return input_fn


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [24]:
#from pathlib import Path
train_file = os.path.join('./working', "train.tf_record")
#filename = Path(train_file)
if not os.path.exists(train_file):
    open(train_file, 'w').close()

In [27]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_examples) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [28]:
file_based_convert_examples_to_features(
            train_examples, MAX_SEQ_LENGTH, tokenizer, train_file)
tf.logging.set_verbosity(tf.logging.INFO)
tf.logging.info("***** Running training *****")
tf.logging.info("  Num examples = %d", len(train_examples))
tf.logging.info("  Batch size = %d", BATCH_SIZE)
tf.logging.info("  Num steps = %d", num_train_steps)

I0709 14:03:29.202362 140073948907264 <ipython-input-28-167d3f1eaaf2>:4] ***** Running training *****
I0709 14:03:29.203487 140073948907264 <ipython-input-28-167d3f1eaaf2>:5]   Num examples = 7452
I0709 14:03:29.204046 140073948907264 <ipython-input-28-167d3f1eaaf2>:6]   Batch size = 32
I0709 14:03:29.204461 140073948907264 <ipython-input-28-167d3f1eaaf2>:7]   Num steps = 4657


In [29]:
train_input_fn = file_based_input_fn_builder(
    input_file=train_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)

In [30]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)
model_fn = model_fn_builder(
  bert_config=bert_config,
  num_labels= len(LABEL_COLUMNS),
  init_checkpoint=BERT_INIT_CHKPNT,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=False,
  use_one_hot_embeddings=False)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

I0709 14:03:37.629574 140073948907264 estimator.py:209] Using config: {'_model_dir': './output', '_tf_random_seed': None, '_save_summary_steps': 500, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f654666ad30>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [31]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

I0709 14:03:40.445046 140073948907264 estimator.py:360] Skipping training since max_steps has already saved.


Beginning Training!
Training took time  0:00:00.004600


In [32]:
eval_file = os.path.join('./working', "eval.tf_record")
#filename = Path(train_file)
if not os.path.exists(eval_file):
    open(eval_file, 'w').close()

eval_examples = create_examples(x_val)
file_based_convert_examples_to_features(
    eval_examples, MAX_SEQ_LENGTH, tokenizer, eval_file)

In [33]:
# This tells the estimator to run through the entire set.
eval_steps = None

eval_drop_remainder = False
eval_input_fn = file_based_input_fn_builder(
    input_file=eval_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)

W0709 14:03:44.095154 140073948907264 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0709 14:03:44.095896 140073948907264 deprecation.py:323] From <ipython-input-23-2242cb8a9639>:168: map_and_batch (from tensorflow.contrib.data.python.ops.batching) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
W0709 14:03:44.096261 140073948907264 deprecation.py:323] From /home/dp8772/.local/lib/python3.7/site-packages/tensorflow/contrib/data/python/ops/batching.py:273: map_and_batch (from tensorflow.python.data.experimental.ops.batching) is deprecated and will be removed in a future versi

In [58]:
output_eval_file = os.path.join("./working", "eval_results_20epcohs_labeled.txt")

with tf.gfile.GFile(output_eval_file, "w") as writer:
    tf.logging.info("***** Eval results *****")
    for key in sorted(result.keys())[:54]:
        tf.logging.info("  %s = %s", LABEL_COLUMNS[int(key)], str(result[key]))
        writer.write("%s = %s\n" % ( LABEL_COLUMNS[int(key)], str(result[key])))

I0710 10:28:17.263550 140073948907264 <ipython-input-58-bc8c03c3d5e7>:4] ***** Eval results *****
I0710 10:28:17.264266 140073948907264 <ipython-input-58-bc8c03c3d5e7>:6]   Hair Loss = 1.0
I0710 10:28:17.264870 140073948907264 <ipython-input-58-bc8c03c3d5e7>:6]   Viscosity = 0.8809818
I0710 10:28:17.265386 140073948907264 <ipython-input-58-bc8c03c3d5e7>:6]   Cleanliness = 0.9870727
I0710 10:28:17.265733 140073948907264 <ipython-input-58-bc8c03c3d5e7>:6]   Item Availability = 1.0
I0710 10:28:17.266055 140073948907264 <ipython-input-58-bc8c03c3d5e7>:6]   Gift Purposes = 0.3247318
I0710 10:28:17.266374 140073948907264 <ipython-input-58-bc8c03c3d5e7>:6]   Texture = 1.0
I0710 10:28:17.266688 140073948907264 <ipython-input-58-bc8c03c3d5e7>:6]   Hair Dryness = 0.83532953
I0710 10:28:17.267009 140073948907264 <ipython-input-58-bc8c03c3d5e7>:6]   Thickness = 0.95653343
I0710 10:28:17.267314 140073948907264 <ipython-input-58-bc8c03c3d5e7>:6]   Ease of Use = 0.6575304
I0710 10:28:17.267622 140073

In [35]:
x_test = x_test.reset_index(drop=True)
predict_examples = create_examples(x_test,False)

In [36]:
predict_examples = create_examples(x_test,False)
test_features = convert_examples_to_features(predict_examples, MAX_SEQ_LENGTH, tokenizer)

I was sent this product for free to try and review, but all of these opinions are my own. This shampoo did seem to help with a dry scalp and some dandruff. However, it also seemed to make my hair more tangled. The smell was also VERY strong! It had almost a Vicks vapor rub smell to it. I enjoy more floral or berry smells, but if you enjoy essential oils you will most likely enjoy this scent :) [This review was collected as part of a promotion.]
I received this from influenster to try along with the full line.i suffer from scalp issues and worry about using new products. I was soo surprised! I love it!! And it doesn't cost an arm and a leg! [This review was collected as part of a promotion.]
I got to this free with the other two products for testing purposes and as far as I got to use it, it was okay. It didn't help that much in terms of dandruff and the smell was a bit overwhelming but it might be because I'm not a huge fan of apricots. I did notice more strands of hair in the shower f

This stuff is great. I mean, it smells a bit medicinal, but just use a nice smelling conditioner afterwards!
Smells horrible.
My only complaint is the smell it's awful the smell lingers on my hair for hours
I like how the smell of this shampoo last, even after 24hours I can still smell the shampoo in my hair. Loving it
This product doesn’t smell like apple at all. It has a very heavy chemical smell compared to the previous times I’ve purchased it. I hope the seller didn’t cut the product with some other shampoo...
It smelled really bad and I had to return it.
love Head & Shoulders!! Everytime I have dandruff I buy Head & Shoulders and the dandruff is gone within a few days. 
The 2in 1 shampoo and conditioner work really well and still make my hair feel soft, unlike alot of other 2 in 1's. 
I do feel a little residue after I shower with the shampoo for several days. But usually I will alternate the Head and Shoulders with my regular shampoo. 
I love the smell of this specific bottle, it

In [54]:
print('Beginning Predictions!')
current_time = datetime.now()

predict_input_fn = input_fn_builder(features=test_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
predictions = estimator.predict(predict_input_fn)
print("Prediction took time ", datetime.now() - current_time)


Beginning Predictions!
Prediction took time  0:00:00.000602


In [55]:
def create_output(predictions):
    probabilities = []
    for prediction in predictions:
        preds = prediction["probabilities"]
        #print (preds)
        probabilities.append(preds.round())
    
    dff = pd.DataFrame(probabilities)
    dff.columns = LABEL_COLUMNS
    
    return dff

In [56]:
output_df = create_output(predictions)

merged_df =  pd.concat([x_test['id'],output_df], axis=1)
#test_result = merged_df.drop(['verbatim'], axis=1)
test_result = merged_df
test_result.to_csv("sample_test.csv", index=False)

I0709 14:32:02.968494 140073948907264 estimator.py:1145] Calling model_fn.
I0709 14:32:04.848906 140073948907264 <ipython-input-18-ad5610f25c15>:40] num_labels:55;logits:Tensor("loss/BiasAdd:0", shape=(?, 55), dtype=float32);labels:Tensor("loss/Cast:0", shape=(?, 55), dtype=float32)
I0709 14:32:05.460782 140073948907264 <ipython-input-18-ad5610f25c15>:99] **** Trainable Variables ****
I0709 14:32:05.462228 140073948907264 estimator.py:1147] Done calling model_fn.


mode: infer probabilities: Tensor("loss/Sigmoid:0", shape=(?, 55), dtype=float32)


I0709 14:32:05.816048 140073948907264 monitored_session.py:240] Graph was finalized.
I0709 14:32:05.817745 140073948907264 saver.py:1280] Restoring parameters from ./output/model.ckpt-4657
I0709 14:32:06.534958 140073948907264 session_manager.py:500] Running local_init_op.
I0709 14:32:06.577902 140073948907264 session_manager.py:502] Done running local_init_op.


In [57]:
test_result.head()

,id,Hair Loss,Viscosity,Environmental,Smell,Length,Formula Change,Scalp Care,Promotions,Ingredients,...,Returning Customer,Instructions,Shine,Price or Value for Money,Volume & Fullness,Design,Portability,Lather,Spray Application,Professional Care
0,23327,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,23379,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,23392,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,23416,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,23424,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
from sklearn.metrics import multilabel_confusion_matrix
import matplotlib.pyplot as plt

In [44]:
multilabel_confusion_matrix(np.array(x_test[LABEL_COLUMNS]), np.array(test_result[LABEL_COLUMNS]).round())

array([[[932,   0],
        [  0,   0]],

       [[916,   0],
        [ 16,   0]],

       [[932,   0],
        [  0,   0]],

       [[495,  42],
        [  6, 389]],

       [[925,   0],
        [  7,   0]],

       [[932,   0],
        [  0,   0]],

       [[389,  50],
        [  8, 485]],

       [[692,  36],
        [ 26, 178]],

       [[907,   0],
        [ 25,   0]],

       [[932,   0],
        [  0,   0]],

       [[915,  17],
        [  0,   0]],

       [[932,   0],
        [  0,   0]],

       [[931,   0],
        [  1,   0]],

       [[932,   0],
        [  0,   0]],

       [[932,   0],
        [  0,   0]],

       [[911,   1],
        [ 17,   3]],

       [[932,   0],
        [  0,   0]],

       [[932,   0],
        [  0,   0]],

       [[923,   0],
        [  9,   0]],

       [[860,   0],
        [ 72,   0]],

       [[932,   0],
        [  0,   0]],

       [[ 40, 892],
        [  0,   0]],

       [[929,   0],
        [  3,   0]],

       [[932,   0],
        [  0, 